In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from pylab import rcParams
import scipy

import Data_Cleaning



# The ***GOALS*** of this script:
## Dealing with each csv file alone:
- Removing unnecessary columns (such as 'Time of Scrape...)
- Removing all duplication
- Adding 'Scale' columns for each non-numeric variable with a **ratio order**
- Filling the missing values with the mean of each column
## Mixing all files into one big DataBase:
- Adding column named **'Origin'** to track the origin of each file and marge all files into one big DataBase
- Removing all duplication that has accrued from margin files
- counting the total number of data we have and go out to drink a bear :)

In [2]:
name_of_job = 'QA'
path_to_folder = f"data files/{name_of_job}/raw data"
df_QA = Data_Cleaning.merge_csv_files_in_folder(path_to_folder)


Before dropping duplicates: (13065, 23)
After dropping all duplicates: (3892, 23)


In [3]:
name_of_job = 'software engineer'
path_to_folder = f"data files/{name_of_job}/raw data"
df_SE = Data_Cleaning.merge_csv_files_in_folder(path_to_folder)


Before dropping duplicates: (3865, 23)
After dropping all duplicates: (1103, 23)


In [4]:
name_of_job = 'data scientist'
path_to_folder = f"data files/{name_of_job}/raw data"
df_DS = Data_Cleaning.merge_csv_files_in_folder(path_to_folder)


Before dropping duplicates: (10364, 23)
After dropping all duplicates: (5002, 23)


In [5]:
name_of_job = 'game development'
path_to_folder = f"data files/{name_of_job}/raw data"
df_GD = Data_Cleaning.merge_csv_files_in_folder(path_to_folder)

df_GD.columns

Before dropping duplicates: (18049, 23)
After dropping all duplicates: (4355, 23)


Index(['Salary Estimate', 'Annual Salary', 'Job Title', 'Experience',
       'Education', 'Position', 'Company Name', 'Location', 'Rating',
       'Is Remote', 'Company Old', 'Company Size', 'Founded',
       'Type of Ownership', 'Industry', 'Sector', 'Revenue',
       'Career Opportunities', 'Comp & Benefits', 'Culture & Values',
       'Senior Management', 'Work Life Balance', 'Time Unit'],
      dtype='object')

In [6]:
name_of_job = 'MARGE'
path_to_folder = f"data files/{name_of_job}"
df_MR = Data_Cleaning.merge_csv_files_in_folder(path_to_folder)
df_MR.columns

Before dropping duplicates: (38943, 25)
After dropping all duplicates: (12332, 25)


Index(['Salary Estimate', 'Annual Salary', 'Job Title', 'Experience',
       'Education', 'Position', 'Company Name', 'Location', 'Rating',
       'Is Remote', 'Company Old', 'Company Size', 'Founded',
       'Type of Ownership', 'Industry', 'Sector', 'Revenue',
       'Career Opportunities', 'Comp & Benefits', 'Culture & Values',
       'Senior Management', 'Work Life Balance', 'Time Unit',
       'Scale_Company_Size', 'Scale_Revenue'],
      dtype='object')

### Preprocessing stage for each df_name:
- Adding 'Scale' columns
- Filling the missing values with 'mean' for each column
- Adding 'one hot encoding' for specific columns ONLY after marge!

In [7]:
# Adding 'Scale' columns
job_list = [df_GD, df_DS, df_SE, df_QA]
list_location = ['Remote', 'United State', 'Michigan', 'Pennsylvania', 'Colorado']

for job in job_list:
    Data_Cleaning.preprocess_scaling(job)



In [13]:
# Filling the missing values with 'mean' for each column. And cleaning the location (agine, only if needed).
for job in job_list:
    Data_Cleaning.filling_missing_values(job)
    job['Location'] = job['Location'].fillna('')
    job['Location'] = job['Location'].apply(lambda x: x.split(',')[-1])
    job['Location'] = job['Location'].apply(lambda x: x if len(x) < 4 or x in list_location else 'LA')
    job['Location'] = job['Location'].apply(lambda x: x if x != '-1' else 'LA')
# print(df_DS.Location.unique())
# print(df_DS['Type of Ownership'].unique())
# print(df_SE.columns)
# df_GD.head()

# Margin the files into one DataBase

In [14]:

jobs_to_connect = [df_SE, df_GD, df_QA, df_DS]
jobs = pd.concat(jobs_to_connect)
print(f"shape before: {jobs.shape}")
jobs.drop_duplicates(keep='first', inplace=True)
print(f"shape after: {jobs.shape}")
jobs.head()

shape before: (14352, 25)
shape after: (13796, 25)


,Salary Estimate,Annual Salary,Job Title,Experience,Education,Position,Company Name,Location,Rating,Is Remote,...,Sector,Revenue,Career Opportunities,Comp & Benefits,Culture & Values,Senior Management,Work Life Balance,Time Unit,Scale_Company_Size,Scale_Revenue
0,$30.00 /hr (est.),62400.0,Web Development Foundations (Complex) || Fresher,5.0,1,1,Hurix Digital\n3.2,Remote,3.200000,1,...,Information Technology,Unknown / Non-Applicable,3.200000,3.000000,3.100000,3.000000,3.000000,hr,4.000000,7.140426
1,$67.50 /hr (est.),140400.0,Cloud Engineer,5.0,2,-1,CloudXoo,Remote,3.915368,1,...,-1,-1,3.738412,3.774893,3.859764,3.574678,3.788948,hr,4.799479,7.140426
2,$19.00 /hr (est.),39520.0,Cyber Security and Cloud Computing Tutor,0.0,1,-1,Georgetown Learning Centers\n4.2,Remote,4.200000,1,...,-1,-1,3.600000,3.500000,3.900000,3.700000,3.300000,hr,4.799479,7.140426
3,$71.50 /hr (est.),148720.0,Cloud Architect,5.0,2,-1,CloudXoo,Remote,3.915368,1,...,-1,-1,3.738412,3.774893,3.859764,3.574678,3.788948,hr,4.799479,7.140426
4,$30.00 /hr (est.),62400.0,Scripting and Programming Foundations (Complex...,5.0,1,1,Hurix Digital\n3.2,Remote,3.200000,1,...,Information Technology,Unknown / Non-Applicable,3.200000,3.000000,3.100000,3.000000,3.000000,hr,4.000000,7.140426


In [16]:
print(jobs.shape)
jobs.head()

(13796, 25)


,Salary Estimate,Annual Salary,Job Title,Experience,Education,Position,Company Name,Location,Rating,Is Remote,...,Sector,Revenue,Career Opportunities,Comp & Benefits,Culture & Values,Senior Management,Work Life Balance,Time Unit,Scale_Company_Size,Scale_Revenue
0,$30.00 /hr (est.),62400.0,Web Development Foundations (Complex) || Fresher,5.0,1,1,Hurix Digital\n3.2,Remote,3.200000,1,...,Information Technology,Unknown / Non-Applicable,3.200000,3.000000,3.100000,3.000000,3.000000,hr,4.000000,7.140426
1,$67.50 /hr (est.),140400.0,Cloud Engineer,5.0,2,-1,CloudXoo,Remote,3.915368,1,...,-1,-1,3.738412,3.774893,3.859764,3.574678,3.788948,hr,4.799479,7.140426
2,$19.00 /hr (est.),39520.0,Cyber Security and Cloud Computing Tutor,0.0,1,-1,Georgetown Learning Centers\n4.2,Remote,4.200000,1,...,-1,-1,3.600000,3.500000,3.900000,3.700000,3.300000,hr,4.799479,7.140426
3,$71.50 /hr (est.),148720.0,Cloud Architect,5.0,2,-1,CloudXoo,Remote,3.915368,1,...,-1,-1,3.738412,3.774893,3.859764,3.574678,3.788948,hr,4.799479,7.140426
4,$30.00 /hr (est.),62400.0,Scripting and Programming Foundations (Complex...,5.0,1,1,Hurix Digital\n3.2,Remote,3.200000,1,...,Information Technology,Unknown / Non-Applicable,3.200000,3.000000,3.100000,3.000000,3.000000,hr,4.000000,7.140426


In [17]:
jobs.to_csv('DataBase_noEncoding_var1.csv', index=False)

In [18]:
# Adding 'one hot encoding' for specific columns

columns = ['Time Unit', 'Type of Ownership', 'Location']  # for one-hot encoding
jobs = Data_Cleaning.one_hot_encoding(jobs, columns)

jobs.shape

(13796, 94)

## Only now we can connect the new data to the data base!

In [19]:
dataBase = pd.read_csv('DataBaseVar/DataBase_var1.csv', index_col=0)
jobs = pd.concat([jobs, dataBase])
print(f"shape before: {jobs.shape}")
jobs.drop_duplicates(keep='first', inplace=True)
print(f"shape after: {jobs.shape}")
jobs.head()

shape before: (28392, 94)
shape after: (28392, 94)


,Salary Estimate,Annual Salary,Job Title,Experience,Education,Position,Company Name,Rating,Is Remote,Company Old,...,Location_ VT,Location_ WA,Location_ WI,Location_ WV,Location_ WY,Location_Colorado,Location_LA,Location_Michigan,Location_Pennsylvania,Location_Remote
0,$30.00 /hr (est.),62400.0,Web Development Foundations (Complex) || Fresher,5.0,1,1,Hurix Digital\n3.2,3.200000,1,23.000000,...,0,0,0,0,0,0,0,0,0,1
1,$67.50 /hr (est.),140400.0,Cloud Engineer,5.0,2,-1,CloudXoo,3.915368,1,49.744186,...,0,0,0,0,0,0,0,0,0,1
2,$19.00 /hr (est.),39520.0,Cyber Security and Cloud Computing Tutor,0.0,1,-1,Georgetown Learning Centers\n4.2,4.200000,1,49.744186,...,0,0,0,0,0,0,0,0,0,1
3,$71.50 /hr (est.),148720.0,Cloud Architect,5.0,2,-1,CloudXoo,3.915368,1,49.744186,...,0,0,0,0,0,0,0,0,0,1
4,$30.00 /hr (est.),62400.0,Scripting and Programming Foundations (Complex...,5.0,1,1,Hurix Digital\n3.2,3.200000,1,23.000000,...,0,0,0,0,0,0,0,0,0,1


In [20]:
jobs.to_csv('DataBase_var2.csv',index=False)